In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc

Came across this blog post about downcasting the data as i tried it found it really cool.
https://medium.com/@alielagrebi/optimize-the-pandas-dataframe-memory-consuming-for-low-environment-24aa74cf9413

In [ ]:
import seaborn as sns

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-oct-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-oct-2021/test.csv')
submission = pd.read_csv('../input/tabular-playground-series-oct-2021/sample_submission.csv')

In [ ]:
def memory_usage_per_type(df):
    types = ['float','int','object']
    for type in types:
        selected_col = df.select_dtypes(include=[type])
        memory_usage_of_selected_type_b = selected_col.memory_usage(deep=True).sum()
        memory_usage_of_selected_type_mb = memory_usage_of_selected_type_b/1024**2
        print("memeory usage for {} columns: {:03.2f} MB".format(type,memory_usage_of_selected_type_mb))
def memory_usage(df):
    return df.memory_usage(deep=True).sum()/1024**2
memory_usage_per_type(train)
print("Memory use before optimization {:.2f} MB".format(memory_usage(train)))

In [ ]:
def downcast_and_compare_memory_consuming_by_type_first(type_):
    selected_type = train.select_dtypes(include=[type_])
    converted_type = selected_type.apply(pd.to_numeric,downcast=type_)
    memory_before = memory_usage(selected_type)
    memory_after = memory_usage(converted_type)
    print('before the downcast:{:.2f} MB'.format(memory_before))
    print('after the downcast:{:.2f} MB'.format(memory_after))
    print('')
    compare_type_changes = pd.concat([selected_type.dtypes,converted_type.dtypes],axis=1)
    compare_type_changes.columns = ['before','after']
    return compare_type_changes.apply(pd.Series.value_counts)

In [ ]:
downcast_and_compare_memory_consuming_by_type_first('float')

In [ ]:
def downcast_and_compare_memory_consuming_by_type_second(type_):
    selected_col = train.select_dtypes(include=[type_])
    memory_before = memory_usage(selected_col)
    optimize_col = selected_col.copy()
    for col in optimize_col.columns:
        mx_col = optimize_col[col].max()
        mn_col = optimize_col[col].min()
        if type_=='int':
            if mn_col>0:
                    if mx_col < 255:
                        optimize_col[col] = optimize_col[col].astype(np.uint8)
                    elif mx_col < 65535:
                        optimize_col[col] = optimize_col[col].astype(np.uint16)
                    elif mx_col < 4294967295:
                        optimize_col[col] = optimize_col[col].astype(np.uint32)
                    else:
                        optimize_col[col] = optimize_col[col].astype(np.uint64)
            else:
                    if mn_col > np.iinfo(np.int8).min and mx < np.iinfo(np.int8).max:
                        optimize_col[col] = optimize_col[col].astype(np.int8)
                    elif mn > np.iinfo(np.int16).min and mx < np.iinfo(np.int16).max:
                        optimize_col[col] = optimize_col[col].astype(np.int16)
                    elif mn > np.iinfo(np.int32).min and mx < np.iinfo(np.int32).max:
                        optimize_col[col] = optimize_col[col].astype(np.int32)
                    elif mn > np.iinfo(np.int64).min and mx < np.iinfo(np.int64).max:
                        optimize_col[col] = optimize_col[col].astype(np.int64)
        else:
            if mn_col > np.finfo(np.float16).min and mx_col < np.finfo(np.float16).max:
                    optimize_col[col] = optimize_col[col].astype(np.float16)
            elif mn_col > np.finfo(np.float32).min and mx_col < np.finfo(np.float32).max:
                    optimize_col[col] = optimize_col[col].astype(np.float32)
            else:
                    optimize_col[col] = optimize_col[col].astype(np.float64)
            
    memory_after = memory_usage(optimize_col)
    print('before the downcast:{:.2f} MB'.format(memory_before))
    print('after the downcast:{:.2f} MB'.format(memory_after))
    compare_type_changes = pd.concat([selected_col.dtypes,optimize_col.dtypes],axis=1)
    compare_type_changes.columns = ['before','after']
    return compare_type_changes.apply(pd.Series.value_counts)

In [ ]:
downcast_and_compare_memory_consuming_by_type_second('float')
